In [ ]:
import numpy as np
from scipy.stats import norm, gamma as gamma_dist, beta as beta_dist
from scipy.special import gamma

# ------------------------------------------------------------
# کلاس پایه (اختیاری)
# ------------------------------------------------------------
class Estimator:
    def estimate(self, data):
        raise NotImplementedError

# ------------------------------------------------------------
# تخمین با روش گشتاوری (Method of Moments)
# ------------------------------------------------------------
class MethodOfMoments(Estimator):
    def __init__(self, distribution):
        """
        distribution: نام توزیع (یک رشته)
        پشتیبانی از: 'normal', 'exponential', 'bernoulli', 'poisson'
        """
        self.distribution = distribution.lower()

    def estimate(self, data):
        data = np.asarray(data)
        n = len(data)
        if self.distribution == 'normal':
            mu = np.mean(data)
            sigma = np.std(data, ddof=0)  # انحراف معیار با تقسیم بر n (گشتاوری)
            return {'mu': mu, 'sigma': sigma}
        elif self.distribution == 'exponential':
            lam = 1.0 / np.mean(data)
            return {'rate': lam}
        elif self.distribution == 'bernoulli':
            p = np.mean(data)
            return {'p': p}
        elif self.distribution == 'poisson':
            lam = np.mean(data)
            return {'lambda': lam}
        else:
            raise ValueError(f"توزیع '{self.distribution}' پشتیبانی نمی‌شود")

# ------------------------------------------------------------
# تخمین با روش درست‌نمایی ماکسیمم (Maximum Likelihood)
# ------------------------------------------------------------
class MaximumLikelihood(Estimator):
    def __init__(self, distribution):
        self.distribution = distribution.lower()

    def estimate(self, data):
        data = np.asarray(data)
        n = len(data)
        if self.distribution == 'normal':
            mu = np.mean(data)
            sigma = np.std(data, ddof=1)  # انحراف معیار نمونه (تقسیم بر n-1) برای نااریب بودن، ولی MLE با تقسیم بر n است
            # توجه: MLE برای واریانس = mean((x-mu)^2) با تقسیم بر n
            # در اینجا sigma_MLE = sqrt(mean((x-mu)^2))
            sigma_mle = np.sqrt(np.mean((data - mu)**2))
            return {'mu': mu, 'sigma': sigma_mle}
        elif self.distribution == 'exponential':
            lam = 1.0 / np.mean(data)  # همان MLE
            return {'rate': lam}
        elif self.distribution == 'bernoulli':
            p = np.mean(data)
            return {'p': p}
        elif self.distribution == 'poisson':
            lam = np.mean(data)
            return {'lambda': lam}
        else:
            raise ValueError(f"توزیع '{self.distribution}' پشتیبانی نمی‌شود")

# ------------------------------------------------------------
# تخمین بیزی (Bayesian) با توزیع‌های مزدوج
# ------------------------------------------------------------
class Bayesian(Estimator):
    def __init__(self, distribution, prior_params):
        """
        distribution: نام توزیع
        prior_params: دیکشنری شامل پارامترهای توزیع پیشین
        - برای نرمال (میانگین با واریانس معلوم): prior_params = {'mu0':, 'sigma0':} (پیشین نرمال برای میانگین)
        - برای نمایی: prior_params = {'alpha':, 'beta':} (پیشین گاما برای نرخ)
        - برای برنولی: prior_params = {'alpha':, 'beta':} (پیشین بتا برای p)
        - برای پواسون: prior_params = {'alpha':, 'beta':} (پیشین گاما برای λ)
        """
        self.distribution = distribution.lower()
        self.prior = prior_params

    def estimate(self, data):
        data = np.asarray(data)
        n = len(data)
        if self.distribution == 'normal':
            # تخمین میانگین با فرض واریانس معلوم (پیشین نرمال)
            mu0 = self.prior.get('mu0', 0)
            sigma0 = self.prior.get('sigma0', 1e6)  # واریانس پیشین (فرض می‌کنیم معلوم)
            sigma_known = self.prior.get('sigma_known', 1)  # انحراف معیار جامعه (معلوم)
            # میانگین داده
            x_bar = np.mean(data)
            # واریانس پسین
            post_var = 1 / (1/sigma0**2 + n/sigma_known**2)
            # میانگین پسین
            post_mean = post_var * (mu0/sigma0**2 + n*x_bar/sigma_known**2)
            return {'mu': post_mean, 'sigma_known': sigma_known}
        elif self.distribution == 'exponential':
            # پیشین گاما: alpha, beta (نرخ = λ)
            alpha0 = self.prior.get('alpha', 1)
            beta0 = self.prior.get('beta', 1)
            alpha_post = alpha0 + n
            beta_post = beta0 + np.sum(data)
            # تخمین نقطه‌ای: میانگین پسین (beta/alpha) برای نرخ؟ دقت: در پارامترسازی گاما به صورت shape=alpha, rate=beta، میانگین = alpha/beta
            # پس میانگین پسین نرخ = alpha_post / beta_post
            rate_mean = alpha_post / beta_post
            return {'rate': rate_mean, 'alpha_post': alpha_post, 'beta_post': beta_post}
        elif self.distribution == 'bernoulli':
            # پیشین بتا: alpha, beta
            alpha0 = self.prior.get('alpha', 1)
            beta0 = self.prior.get('beta', 1)
            successes = np.sum(data)
            alpha_post = alpha0 + successes
            beta_post = beta0 + n - successes
            # تخمین نقطه‌ای: میانگین پسین = alpha/(alpha+beta)
            p_mean = alpha_post / (alpha_post + beta_post)
            return {'p': p_mean, 'alpha_post': alpha_post, 'beta_post': beta_post}
        elif self.distribution == 'poisson':
            # پیشین گاما: alpha, beta (برای λ)
            alpha0 = self.prior.get('alpha', 1)
            beta0 = self.prior.get('beta', 1)
            alpha_post = alpha0 + np.sum(data)
            beta_post = beta0 + n
            lambda_mean = alpha_post / beta_post
            return {'lambda': lambda_mean, 'alpha_post': alpha_post, 'beta_post': beta_post}
        else:
            raise ValueError(f"توزیع '{self.distribution}' پشتیبانی نمی‌شود")